In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import random
import os
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
import fasttext
import certifi
import ssl
import urllib.request
import os
import gzip
import shutil
import subprocess
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
# andom.seed(seed)
# np.random.seed(seed)seed = 2024
# # 固定种子，结果可重复


In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [3]:
import re
def remove_urls(text):
    # 定义URL的正则表达式
    url_pattern = r'https?://\S+|www\.\S+'
    # 使用re.sub()函数替换URL为空字符串
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text

In [4]:
file['content'] = file['content'].apply(remove_urls) 
file['content'] = file['content'].apply(clean_text) 
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content']
file.head()

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_7340/659970310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_7340/659970310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_7340/659970310.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,content,sentiment
0,om het nog bondiger te zeggen ik stemde ooit l...,1
1,volgens de peilingen stond pvv weken geleden ...,1
2,pvv stemmers mogen dan het geheugen hebben van...,1
3,pvv wil geen vluchtelingen omtzigt wil geen ex...,2
4,dat weten we niet pvv heeft veel strategische ...,2


In [5]:
import fasttext

# Path to your FastText pre-trained word embedding file
fasttext_model_path = '/Users/rachael/Downloads/cc.nl.300.bin'

# Load the FastText model
ft = fasttext.load_model(fasttext_model_path)

# # Example usage: Get the vector for a word
# word = "voorbeeld"
# word_vector = ft.get_word_vector(word)

# print(f"Vector for the word '{word}':\n{word_vector}")

In [6]:
import spacy
nlp = spacy.load("nl_core_news_sm")
#tokenize
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

def sentence_to_avg_vec(tokens, model):
    vector_size = model.get_dimension()
    vec = np.zeros(vector_size)
    count = 0
    for token in tokens:
        vec += model.get_word_vector(token)
        count += 1
    if count != 0:
        vec /= count
    return vec

In [7]:
param_dist = {
    'C': np.logspace(-4, 4, 20),  # Inverse of regularization strength
    'solver': ['newton-cg', 'lbfgs', 'saga'],  # Solvers that support multinomial loss
    'max_iter': [20000, 30000,40000],  # Maximum number of iterations
}

best_score = 0
best_params = {}

test_scores = []
best_params_list = []
f1_scores = []

In [8]:
# Stratified K-fold for maintaining label distribution,shuffle=True确保每次迭代的数据划分不同
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Loop over each fold
for train_index, test_index in outer_cv.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]
   
    y_train_df = pd.DataFrame(trainy).reset_index(drop=True)
    y_test_df = pd.DataFrame(testy).reset_index(drop=True)


    train_df = pd.concat([trainX.reset_index(drop=True), y_train_df], axis=1)
    test_df = pd.concat([testX.reset_index(drop=True), y_test_df], axis=1)
 
    train_df['content'] = train_df['content'].apply(lambda x: ' '.join(x.split()))
    sentiment_0 = train_df[train_df['sentiment'] == 0]
    sentiment_1 = train_df[train_df['sentiment'] == 1]
    sentiment_2 = train_df[train_df['sentiment'] == 2]

    output_file_path0 = '/Users/rachael/Desktop/data/sentiment_0.txt'
    with open(output_file_path0, 'w', encoding='utf-8') as f:
        for _, row in sentiment_0.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")     
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path0,
    '--output', '/Users/rachael/Desktop/data/augmented_train0.txt',  # Output as txt if required
    '--num_aug', '9',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])

    output_file_path1 = '/Users/rachael/Desktop/data/sentiment_1.txt'
    with open(output_file_path1, 'w', encoding='utf-8') as f:
        for _, row in sentiment_1.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path1,
    '--output', '/Users/rachael/Desktop/data/augmented_train1.txt',  # Output as txt if required
    '--num_aug', '1',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])
    

    output_file_path2 = '/Users/rachael/Desktop/data/sentiment_2.txt'
    with open(output_file_path2, 'w', encoding='utf-8') as f:
        for _, row in sentiment_2.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path2,
    '--output', '/Users/rachael/Desktop/data/augmented_train2.txt',  # Output as txt if required
    '--num_aug', '4',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])
    # Load augmented data
    file4 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train0.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    file5 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train1.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    file6 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train2.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    augmented_train = pd.concat([file4,file5,file6])

    augmented_train['content'] = augmented_train['content'].astype(str)
    train_sentences = [item for item in list(augmented_train['content'])]
    train_labels = [int(item) for item in list(augmented_train['sentiment'])]
    X_train_tokenized = [tokenize(sentence) for sentence in train_sentences]
    train_vectors = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_train_tokenized])

    test_df['content'] = test_df['content'].astype(str)
    test_sentences = [item for item in list(test_df['content'])]
    test_labels = [int(item) for item in list(test_df['sentiment'])]
    X_test_tokenized = [tokenize(sentence) for sentence in test_sentences]
    test_vectors = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_test_tokenized])

    
    lr = LogisticRegression(multi_class='multinomial', random_state=42,class_weight='balanced')
    randomized_search = RandomizedSearchCV(
        estimator=lr,
        param_distributions=param_dist,
        n_iter=100,  # Number of parameter settings that are sampled
        scoring='f1_macro',
        n_jobs=-1,  # Use all available cores
        cv=inner_cv,
        random_state=42,
        verbose=1
    )

    # Fit Randomized Search
    randomized_search.fit(train_vectors, train_labels)

    # Get the best model from Randomized Search
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # Evaluate on the test set
    test_predictions = best_model.predict(test_vectors)
    test_score = accuracy_score(test_labels, test_predictions)
    test_f1_score = f1_score(test_labels, test_predictions, average='macro')

    test_scores.append(test_score)
    f1_scores.append(test_f1_score)
    best_params_list.append(best_params)

    print(f"Fold test score: {test_score}")
    print(f"Fold F1-score: {test_f1_score}")
    print(f"Best parameters: {best_params}")

# Summarize the results
mean_test_score = np.mean(test_scores)
std_test_score = np.std(test_scores)
mean_f1_score = np.mean(f1_scores)
std_f1_score = np.std(f1_scores)

print(f"Mean test score: {mean_test_score}")
print(f"Standard deviation of test scores: {std_test_score}")
print(f"Mean F1-score: {mean_f1_score}")
print(f"Standard deviation of F1-scores: {std_f1_score}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_0.txt to /Users/rachael/Desktop/data/augmented_train0.txt with num_aug=9
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_1.txt to /Users/rachael/Desktop/data/augmented_train1.txt with num_aug=1
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_2.txt to /Users/rachael/Desktop/data/augmented_train2.txt with num_aug=4
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold test score: 0.5487804878048781
Fold F1-score: 0.40448207898386745
Best parameters: {'solver': 'lbfgs', 'max_iter': 20000, 'C': 10000.0}
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_0.txt to /Users/rachael/Desktop/data/augmented_train0.txt with num_aug=9
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_1.txt to /Users/rachael/Desktop/data/augmented_train1.txt with num_aug=1
generated augmented senten

In [9]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'C': 10000.0, 'max_iter': 20000, 'solver': 'lbfgs'}


In [7]:
most_common_params = {'solver': 'lbfgs', 'max_iter': 20000, 'C': 10000.0}


In [8]:
X_sentences = [item for item in list(X)]
X_tokenized = X.apply(tokenize)
X_embeddings = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_tokenized])

final_model = LogisticRegression(multi_class='multinomial', random_state=42, **most_common_params,class_weight='balanced')
final_model.fit(X_embeddings, y)

LogisticRegression(C=10000.0, class_weight='balanced', max_iter=20000,
                   multi_class='multinomial', random_state=42)

In [9]:
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
#the data verified by the third person will be used as the train set finally
test = test[['content','sentiment']]
test['content'] = test['content'].apply(remove_urls) 
test['content'] = test['content'].apply(clean_text) 
test['sentiment'] = test['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
test_tokenized = test['content'].apply(tokenize)
test_embedding = np.array([sentence_to_avg_vec(sentence,ft) for sentence in test_tokenized])

test_predictions = final_model.predict(test_embedding)
test_score = accuracy_score(test['sentiment'], test_predictions)
conf_matrix = confusion_matrix(test['sentiment'], test_predictions)
class_report = classification_report(test['sentiment'], test_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.06      0.08      0.07        38
           1       0.67      0.74      0.71       245
           2       0.51      0.34      0.41       124

    accuracy                           0.56       407
   macro avg       0.41      0.39      0.39       407
weighted avg       0.57      0.56      0.56       407

